# Дисклеймер
Эту тетрадку нужно запускать в колабе или в vast.ai. Не мучатесь с установкой библиотек и с обучением на cpu.

In [1]:
!pip install tokenizers matplotlib sklearn

     |████████████████████████████████| 3.3 MB 2.1 MB/s eta 0:00:01     |████████████████████████▊       | 2.5 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 22.2 MB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 7.3 MB/s eta 0:00:012
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=fc1aadbaca430e2fd6e2815b7c36a7f6106f1d5833509ef78ee4aaf5a450b019
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
# в vast ai или в последних версия jupyter может не работать автозаполнение, установка этой либы и перезагрука кернела помогает
# !pip install --upgrade jedi==0.17.2

# Транформеры для решения seq2seq задач

Seq2seq - наверное самая общая формальная постановка задачи в NLP. Нужно из произвольной последовательности получить какую-то другую последовательность. И в отличие от разметки последовательности (sequence labelling) не требуется, чтобы обе последовательности совпадали по длине. Даже стандартную задачу классификации можно решать как seq2seq - можно рассматривать метку класса как последовательность длинны 1.

А трансформеры - sota архитектура для seq2seq задач. Мы не будем подробно разбирать устройство транформеров, если вам интересно вы можете поразбираться вот с этими материалами:

Оригинальная статья (сложновато) - https://arxiv.org/pdf/1706.03762.pdf

https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/  
https://jalammar.github.io/illustrated-transformer/

https://www.youtube.com/watch?v=iDulhoQ2pro

https://www.youtube.com/watch?v=TQQlZhbC5ps



Трансформеры будут подробно разбираться на курсе глубокого обучения на втором курсе.

Пока просто попробуем обучать модель на задаче машинного перевода. Для таких задач лучше всего использовать предобученные модели, но если у вас будет какая-то специфичная seq2seq задача, то имеет смысл попробовать обучить трансформер с нуля и в этой тертрадке вам нужно будет поменять только часть с загрузкой данных. 

In [159]:

import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [160]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
# text = open('opus.en-ru-train.ru.txt').read().replace('\xa0', ' ')
# f = open('opus.en-ru-train.ru.txt', 'w')
# f.write(text)
# f.close()

Данные взяты вот отсюда - https://opus.nlpl.eu/opus-100.php (раздел с отдельными языковыми парами)

In [161]:
en_sents = open('opus.en-ru-train.en.txt').read().splitlines()
ru_sents = open('opus.en-ru-train.ru.txt').read().splitlines()

Пример перевода с английского на русский

In [162]:
en_sents[-1], ru_sents[-1]

('So what are you thinking?', 'Ну и что ты думаешь?')

Как обычно нам нужен токенизатор, а точнее даже 2, т.к. у нас два корпуса

In [163]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=["opus.en-ru-train.en.txt"], trainer=trainer_en)

tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_ru.train(files=["opus.en-ru-train.ru.txt"], trainer=trainer_ru)

### ВАЖНО!

Токенизатор - это неотъемлимая часть модели, поэтому не забывайте сохранять токенизатор вместе с моделью. Если вы забудете про это и переобучите токенизатор, то индексы токенов разойдутся и веса модели будут бесполезны. 

In [ ]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
# tokenizer_en.save('tokenizer_en')
# tokenizer_ru.save('tokenizer_ru')

In [168]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

Переводим текст в индексы вот таким образом. В начало добавляем токен '[CLS]', а в конец '[SEP]'. Если вспомните занятие по языковому моделированию, то там мы добавляли "\<start>" и "\<end>" - cls и sep по сути тоже самое. Вы поймете почему именно cls и sep, а не start и end, если подробнее поразбираетесь с устройством трансформеров

In [169]:
def encode(text, tokenizer):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids + [tokenizer.token_to_id('[SEP]')]

Кодируем и паддим

In [170]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru) for t in ru_sents]

In [171]:
max_len_en = np.mean([len(x) for x in X_en])
max_len_ru = np.mean([len(x) for x in X_ru])

In [172]:
max_len_en, max_len_ru

(17.256243, 17.059659)

In [ ]:
# ограничимся длинной в 20 и 22 (разные чтобы показать что в seq2seq не нужна одинаковая длина)
max_len_en, max_len_ru = 20, 22

In [181]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

3

In [227]:
X_en = tf.keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=max_len_en, padding='post', value=float(PAD_IDX))
X_ru = tf.keras.preprocessing.sequence.pad_sequences(
              X_ru, maxlen=max_len_ru, padding='post', value=float(PAD_IDX))

In [228]:
# миллион примеров 
X_en.shape, X_ru.shape

((1000000, 20), (1000000, 22))

Разделяем на трейн и тест

In [229]:
X_en_train, X_en_valid, X_ru_train, X_ru_valid = train_test_split(X_en, X_ru, test_size=0.05)

Дальше код модели, он взят вот отсюда (с небольшими изменениями) - https://www.tensorflow.org/text/tutorials/transformer

Там есть комментарии по каждому этапу

# Transformer

In [230]:
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

In [231]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

In [232]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, PAD_IDX), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [233]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [234]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):

        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [235]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [236]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [237]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [238]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [239]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
      vocab_size=vocab_size[0],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[0],
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
      vocab_size=vocab_size[1],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[1],
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [240]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, max_len_ru - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

Определяем параметры модели. Есть стандартные наборы параметров для обучения маленькой, средней или большой модели. Их можно посмотреть в оригинальной статье про трансформер - https://arxiv.org/pdf/1706.03762.pdf

Мы возьмем маленький, т.к. он требует меньше ресурсов. НО для практической задачи есть смысл использовать модель побольше, есть статьи в которые показано, что большие трансформеры обучать эффективнее (требуется меньше примеров) -  https://arxiv.org/pdf/2002.11794.pdf.

In [242]:
tf.keras.backend.clear_session()

# small model
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1


# average model
# NUM_LAYERS = 6
# D_MODEL = 512
# NUM_HEADS = 8
# UNITS = 2048
# DROPOUT = 0.1


# Если доступно несколько гпу
# расскоментируйте и сдвиньте остальной код ниже на 1 таб
# mirrored_strategy = tf.distribute.MirroredStrategy()
# with mirrored_strategy.scope():
model = transformer(
    vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    max_len=[max_len_en, max_len_ru])

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    0.0001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, max_len_ru - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_ruen',
                                            monitor='val_loss',
                                            verbose=1,
                                        save_weights_only=True,
                                        save_best_only=True,
                                        mode='min',
                                        save_freq='epoch')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


Трансформеры - большие модели, обучаться они должны долго. Размер батча здесь уже имеет большое значение: сликом маленький батч приведет к сильному увеличению времени на обучение, но сильно большим его поставить не получится, т.к. модель большая и быстро займет всю видеокарту.

In [253]:
# Обратите внимание на то как данные подаются в модель (в частности на индексы [:, :-1] и [:,1:])
# помимо текста в модель еще нужно передать целевую последовательность
# но не полную а без 1 последнего элемента
# а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
# мы сравниваем выход из модели с целевой последовательностью уже с этим последним элементом


model.fit((X_en_train, X_ru_train[:, :-1]), X_ru_train[:,1:], 
             validation_data=((X_en_valid, X_ru_valid[:, :-1]), X_ru_valid[:,1:]),
             batch_size=400,
             epochs=100,
             callbacks=[checkpoint]
             )
    
    
    

Epoch 1/100
2375/2375 [==============================] - 182s 76ms/step - loss: 1.3964 - accuracy: 0.7290 - val_loss: 1.5411 - val_accuracy: 0.7182

Epoch 00001: val_loss improved from 1.55525 to 1.54108, saving model to model_ruen
Epoch 2/100
2375/2375 [==============================] - 182s 77ms/step - loss: 1.3769 - accuracy: 0.7313 - val_loss: 1.5348 - val_accuracy: 0.7189

Epoch 00002: val_loss improved from 1.54108 to 1.53475, saving model to model_ruen
Epoch 3/100
2375/2375 [==============================] - 182s 77ms/step - loss: 1.3592 - accuracy: 0.7335 - val_loss: 1.5281 - val_accuracy: 0.7203

Epoch 00003: val_loss improved from 1.53475 to 1.52812, saving model to model_ruen
Epoch 4/100
2375/2375 [==============================] - 183s 77ms/step - loss: 1.3430 - accuracy: 0.7356 - val_loss: 1.5193 - val_accuracy: 0.7211

Epoch 00004: val_loss improved from 1.52812 to 1.51933, saving model to model_ruen
Epoch 5/100
2375/2375 [==============================] - 182s 77ms/step 

KeyboardInterrupt: 

Можно попробовать перевести рандомное предложение и оценить качество модели самому

In [254]:
def translate(text):
    input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids + [tokenizer_en.token_to_id('[SEP]')]

    input_ids = tf.keras.preprocessing.sequence.pad_sequences(
                                      [input_ids], maxlen=max_len_en, padding='post', value=float(PAD_IDX))

    output_ids = [tokenizer_ru.token_to_id('[CLS]')]

    output_ids_pad = tf.keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru, padding='post', value=float(PAD_IDX))

    pred = model((input_ids, output_ids_pad), training=False)

    while pred.numpy().argmax(2)[0,-1] not in [tokenizer_ru.token_to_id('[SEP]'), tokenizer_ru.token_to_id('[PAD]')]:
        
        output_ids.append(pred.numpy().argmax(2)[0,-1])
        output_ids_pad = tf.keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru, padding='pre')
        pred = model((input_ids, output_ids_pad), training=False)

    return ''.join([tokenizer_ru.id_to_token(i) for i in output_ids[1:]])

In [255]:
translate("Example")

'Пример'

In [256]:
translate("Can you translate this?")

'Вы'